In [1]:
#Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew

In [2]:
# Load training and test datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
# Fill missing categorical columns with 'None'
cat_fill_cols = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu']
for col in cat_fill_cols:
    train_data[col].fillna('None', inplace=True)
    test_data[col].fillna('None', inplace=True)

# Fill numerical columns with median
num_fill_cols = ['LotFrontage', 'GarageYrBlt']
for col in num_fill_cols:
    train_data[col].fillna(train_data[col].median(), inplace=True)
    test_data[col].fillna(test_data[col].median(), inplace=True)

# Fill any remaining missing values with mode
for column in train_data.columns:
    if train_data[column].isnull().sum() > 0:
        train_data[column].fillna(train_data[column].mode()[0], inplace=True)

for column in test_data.columns:
    if test_data[column].isnull().sum() > 0:
        test_data[column].fillna(test_data[column].mode()[0], inplace=True)


In [4]:
# Total square footage
train_data['TotalSqFt'] = train_data['TotalBsmtSF'] + train_data['1stFlrSF'] + train_data['2ndFlrSF']
test_data['TotalSqFt'] = test_data['TotalBsmtSF'] + test_data['1stFlrSF'] + test_data['2ndFlrSF']

# House age at time of sale
train_data['BuildingAge'] = train_data['YrSold'] - train_data['YearBuilt']
test_data['BuildingAge'] = test_data['YrSold'] - test_data['YearBuilt']


In [7]:
# Get all numeric features except the target
numeric_feats = train_data.select_dtypes(include=[np.number]).columns
numeric_feats = numeric_feats.drop('SalePrice')

# Identify skewed features
skew_stats = train_data[numeric_feats].apply(lambda x: skew(x.dropna()))
skewed_cols = skew_stats[abs(skew_stats) > 0.75].index

# Apply log1p transformation
for feature in skewed_cols:
    train_data[feature] = np.log1p(train_data[feature])
    test_data[feature] = np.log1p(test_data[feature])


In [8]:
# Ordinal encoding using LabelEncoder for important quality-based features
ordinal_cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual']
for col in ordinal_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col].astype(str))
    test_data[col] = le.transform(test_data[col].astype(str))

# One-hot encode all remaining categorical features
train_encoded = pd.get_dummies(train_data)
test_encoded = pd.get_dummies(test_data)

# Align both datasets to have the same columns
train_encoded, test_encoded = train_encoded.align(test_encoded, join='left', axis=1)


In [9]:
print("Final Processed Training Data (train_encoded):")
print(train_encoded.head())

print("\Final Processed Test Data (test_encoded):")
print(test_encoded.head())


Final Processed Training Data (train_encoded):
   Id  MSSubClass  LotFrontage   LotArea  OverallQual  OverallCond  YearBuilt  \
0   1    4.110874     1.646667  9.042040            7            5       2003   
1   2    3.044522     1.685370  9.169623            6            8       1976   
2   3    4.110874     1.655196  9.328212            7            5       2001   
3   4    4.262680     1.631370  9.164401            7            5       1915   
4   5    4.110874     1.694266  9.565284            8            5       2000   

   YearRemodAdd  MasVnrArea  ExterQual  ...  SaleType_ConLw  SaleType_New  \
0          2003    5.283204          2  ...               0             0   
1          1976    0.000000          3  ...               0             0   
2          2002    5.093750          2  ...               0             0   
3          1970    0.000000          3  ...               0             0   
4          2000    5.860786          2  ...               0             0   

   

In [10]:
print(f"\ntrain_encoded shape: {train_encoded.shape}")
print(f"test_encoded shape: {test_encoded.shape}")



train_encoded shape: (1460, 277)
test_encoded shape: (1459, 277)
